This notebook uses iFeatureOmega, a feature generation software, to add to our feature space for a RandomForestClassifier that predicts protein pair functionality.

To do:

1) Write unit tests for iFeatureOmega
2) time trial for different descriptors - DONE
3) Figure out how to append meso and thermo descriptors

In [172]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy.stats
import sklearn.preprocessing
import sklearn.model_selection
import sklearn.ensemble
import sklearn.feature_selection
import unittest
import iFeatureOmegaCLI
import Bio.SeqIO
import Bio.SeqRecord
import io
from io import StringIO
import time

In [173]:
cd /Users/loganroberts/Learn2Therm/ValidProt/FAFSA

/Users/loganroberts/Learn2Therm/ValidProt/FAFSA


In [174]:
#convert to pandas df
df = pd.read_csv('learn2therm_sample_50k.csv')
df.columns

Index(['Unnamed: 0', 'local_gap_compressed_percent_id',
       'scaled_local_query_percent_id', 'scaled_local_symmetric_percent_id',
       'query_align_len', 'query_align_cov', 'subject_align_len',
       'subject_align_cov', 'bit_score', 'thermo_index', 'meso_index',
       'prot_pair_index', 'meso_protein_int_index', 'thermo_protein_int_index',
       'taxa_pair_index', 'local_gap_compressed_percent_id_16s',
       'scaled_local_query_percent_id_16s',
       'scaled_local_symmetric_percent_id_16s', 'query_align_cov_16s',
       'subject_align_cov_16s', 'bit_score_16s', 'm_ogt', 't_ogt',
       'ogt_difference', 'm_protein_seq', 't_protein_seq', 'm_protein_desc',
       't_protein_desc', 'm_protein_len', 't_protein_len'],
      dtype='object')

In [175]:
cd /Users/loganroberts/Learn2Therm/ValidProt/notebooks

/Users/loganroberts/Learn2Therm/ValidProt/notebooks


In [176]:
target = pd.read_csv('protein_match_6k.csv')

In [177]:
target

,prot_pair_index,Jaccard_Score,protein_match
0,48641291,1.0,Yes
1,92992745,1.0,Yes
2,157628663,1.0,Yes
3,136708305,1.0,Yes
4,133672542,1.0,Yes
...,...,...,...
6343,55489429,0.5,Yes
6344,172293605,1.0,Yes
6345,47082975,1.0,Yes
6346,154513027,1.0,Yes


In [178]:
df = pd.merge(df, target, on=['prot_pair_index'])
df.shape

(6348, 32)

In [179]:
df

,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,thermo_index,...,t_ogt,ogt_difference,m_protein_seq,t_protein_seq,m_protein_desc,t_protein_desc,m_protein_len,t_protein_len,Jaccard_Score,protein_match
0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,875,...,50.0,22.5,MAESGTSRRADHLVPVPGPDAEPPAVADELLRAVGRGDEQAFGRLY...,MPSQITESERIELAERFERDALPLLDQLYSAALRMTRNPADAEDLV...,ECF RNA polymerase sigma factor SigK,sigma-70 family RNA polymerase sigma factor,206,202,1.0,Yes
1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,11324,...,54.0,29.0,MARIALVDDDRNILTSVSMTLEAEGFEVETYNDGQSALDAFNKRMP...,MRVLLVEDDPNTSRSIEMMLTHANLNVYATDMGEEGIDLAKLYDYD...,response regulator transcription factor,response regulator transcription factor,233,237,1.0,Yes
2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,875,...,50.0,22.0,MKDTVVFVTGAARGIGAHTARLAVARGARVALVGLEPHLLADLAAE...,MTPEQIFSGQTAIVTGGASGIGAATVEHIARRGGRVFSVDLSYDSP...,SDR family oxidoreductase,SDR family oxidoreductase,287,252,1.0,Yes
3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,875,...,50.0,22.0,MTSGLWERVLDGVWVTIQLLVLSALLATAVSFVVGIARTHRLWIVR...,MAMSRRKRGQLARGIQYAILVIVVVVLALLADWGKIGKAFFDWEAA...,ectoine/hydroxyectoine ABC transporter permeas...,amino acid ABC transporter permease,234,269,1.0,Yes
4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,9827,...,50.0,20.0,MIISLRRGLRFIRFIVFFAALVYLFYHVLDLFNGWISPVDQYQMPT...,MKRMVWRTLKVFIIFIACTLLFYFGLRFMHLEYEQFHRYEPPEGPA...,YqzK family protein,YqzK family protein,80,66,1.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,6495,0.387574,0.367978,0.360882,336,0.943820,337,0.910811,588,11707,...,50.0,20.0,MDNEKPIVDFSALRDELGKNYEPVQVMDGQAKVIKPEIVSKYSNDQ...,MKLEFPILQIVDENGNLVNENYRKAITEELVKKFYYHMIRIRTFDR...,pyruvate dehydrogenase (acetyl-transferring) E...,pyruvate dehydrogenase (acetyl-transferring) E...,370,356,0.5,Yes
6344,6496,0.488462,0.488462,0.455197,258,0.992308,263,0.882550,619,875,...,50.0,20.0,MANPYLDDLRLAHVLADSADNLSMDRFGALDLEVSTKPDMTYVTES...,MTRPYADDIHLAKRLADTADAITVPRYSVRDLTVRTKPDRSPVTDA...,histidinol phosphatase,histidinol-phosphatase,298,260,1.0,Yes
6345,6497,0.383838,0.376238,0.374384,296,0.976898,299,0.977124,436,5381,...,45.0,21.5,MGHDHDHSHGTATTNRTKLAWAFGITFTILIAEVIGAILTNSLALL...,MAHQHGPARPEHASGRYLKRLMAAFGIGLVFMVLEVVVGVLTGSLA...,cation diffusion facilitator family transporter,cation diffusion facilitator family transporter,306,303,1.0,Yes
6346,6498,0.769231,0.769231,0.769231,78,1.000000,78,1.000000,258,14963,...,52.5,22.5,MAKPALRKPKKKSNPLKAAKISYVDYKDTALLRKFISDRGKIRARR...,MAKQPPRKPKKKVCVFCQEKISYVDYKDTALLRKFISDRGKIRARR...,30S ribosomal protein S18,30S ribosomal protein S18,78,78,1.0,Yes


In [180]:
def get_fasta_from_dataframe(dataframe, output_file_a, output_file_b):
    #adjust this to write function with BioPython
    #separate functions for each of the input sequences
    #in training, seq_a = meso and seq_b = thermo
    
    
    #meso sequence to fasta
    with open(output_file_a, 'w') as f:
        for _, row in df.iterrows():
            f.write('>{}\n{}\n'.format((row['prot_pair_index']), row['m_protein_seq']))
    
    #thermo sequence to fasta
    with open(output_file_b, 'w') as f:
        for _, row in df.iterrows():
            f.write('>{}\n{}\n'.format((row['prot_pair_index']), (row['t_protein_seq'])))
   
    #return output files
    return [output_file_a, output_file_b]

In [181]:
#getting an error in the try/except block


from Bio import Seq, SeqIO

def get_fasta_from_dataframe_biopython(dataframe, output_file:str):
    # Create a list of SeqRecord objects
    records = []
    for _, row in dataframe.iterrows():
        try:
            seq = Seq(row['m_protein_seq'])
        except Exception as e:
            print(f"Skipping row {row.name}: {str(e)}")
            continue
        record = SeqIO.SeqRecord(
            seq=seq,
            id=row['meso_index'],
            description=""
        )
        records.append(record)
    
    # Write the records to a FASTA file
    with open(output_file, 'w') as f:
        SeqIO.write(records, f, 'fasta')
    
    return output_file


In [182]:
def get_protein_descriptors(fasta_file, descriptors=[]):
    
    """
    Generates features from a protein sequence

    Parameters
    ----------
    Fasta file with amino acid sequences.

    Returns
    -------
    Vector of descriptors
    """
    
    #create iProtein object
    protein = iFeatureOmegaCLI.iProtein(fasta_file)
    
    #not sure why we need this yet. Right now it is stored in local directory.
    params = protein.import_parameters('protein_parameters.json')
    
    protein_descriptors = {}
    
    for descriptor in descriptors:
        protein.get_descriptor(descriptor)
        protein_descriptors.update({f'{descriptor}':protein.encodings})
        
    return protein_descriptors

In [183]:
# def compute_descriptor_difference(fasta_files, descriptors=[]):
        
#     desc_a = get_protein_descriptors(fasta_files[0], descriptors)
#     desc_b = get_protein_descriptors(fasta_files[1], descriptors)
    
#     desc_dict = {}

#     for key in desc_a:
#         desc_dict[key] = desc_a[key] - desc_b[key]
    
#     return desc_dict

In [184]:
def create_new_dataframe(dataframe, output_files, descriptors=[]):
    """
    Creates new dataframe with descriptors added.

    Parameters
    ----------
    Pandas dataframe, list of descriptors as strings, output file name.

    Returns
    -------
    Dataframe including vector(s) of descriptors
    """

    fasta_files = get_fasta_from_dataframe(dataframe, output_files[0], output_files[1])
    
    def compute_descriptor_difference(fasta_files, descriptors=[]):
        """
        Generates dictionary of descriptors for each of the two input sequences.
        Computes the difference between each instance of a descriptor.
        
        Parameters
        ----------
        List of two fasta files (str) and list of descriptors (str).

        Returns
        -------
        Dictionary with difference between descriptors for each of the 
        input sequences.
        """
        desc_a = get_protein_descriptors(fasta_files[0], descriptors)
        desc_b = get_protein_descriptors(fasta_files[1], descriptors)

        feature_dict = {}

        for key in desc_a:
            feature_dict[key] = desc_a[key] - desc_b[key]

        return feature_dict
    
    feature_dict = compute_descriptor_difference(fasta_files, descriptors)


    df = dataframe.reset_index()

    for desc in descriptors:

        feature_dict[desc].index = feature_dict[desc].index.astype(int)
        features = feature_dict[desc].reset_index()

        df = pd.merge(
            df,
            features,
            how='outer',
            left_index=True,
            right_index=True)

    return df

In [185]:
df = create_new_dataframe(df, ['seq_50k_a.fasta', 'seq_50k_b.fasta'], descriptors=['AAC', 'GAAC'])
df

File imported successfully.
File imported successfully.


,index_x,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,...,AAC_T,AAC_V,AAC_W,AAC_Y,index,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
0,0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,...,-0.000961,0.062242,-0.000192,-0.010478,48641291,0.065318,-0.025810,0.011583,-0.017639,-0.033452
1,1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,...,-0.016081,0.018254,0.000072,0.000507,92992745,-0.014143,0.009525,-0.010051,0.032651,-0.017982
2,2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,...,-0.007743,0.011808,0.002516,-0.018293,157628663,0.087205,-0.020132,0.040650,0.006291,-0.114015
3,3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,...,-0.011915,0.019668,0.018762,-0.015251,136708305,0.015966,0.014457,0.001080,-0.011359,-0.020144
4,4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,...,-0.005303,0.042045,0.022348,0.017045,133672542,0.091667,-0.027273,-0.084470,-0.031061,0.051136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,6343,6495,0.387574,0.367978,0.360882,336,0.943820,337,0.910811,588,...,-0.004616,0.011388,-0.006043,0.003811,55489429,0.004783,-0.017871,-0.019040,0.008624,0.023504
6344,6344,6496,0.488462,0.488462,0.455197,258,0.992308,263,0.882550,619,...,-0.011693,-0.033867,-0.001058,0.000903,172293605,-0.039210,0.011306,-0.000800,0.021631,0.007073
6345,6345,6497,0.383838,0.376238,0.374384,296,0.976898,299,0.977124,436,...,0.029023,-0.056753,-0.003397,-0.013266,47082975,-0.035009,-0.017019,0.008801,0.002362,0.040866
6346,6346,6498,0.769231,0.769231,0.769231,78,1.000000,78,1.000000,258,...,-0.051282,0.000000,0.000000,0.000000,154513027,0.064103,-0.012821,0.012821,-0.025641,-0.038462


In [186]:
#this function is only necessary if we get a fasta file with a descriptor generated from component 3

def remove_fasta_description(filename:str):
    
    """
    Removes description from fasta file so that iProtein can read the input.
    Might not be necessary in our current format.
    
    Parameters
    ----------
    Fasta file with protein sequences.

    Returns
    -------
    Fasta file with protein sequences, sans description.
    """
    
    #assign unwanted string to object
    string_to_remove = "<unknown description>"

    #open file
    with open(filename, "r") as file:
        content = file.read()
    
    # Remove the string
    new_content = content.replace(string_to_remove, "")

    #overwrite file without string
    with open(filename, "w") as file:
        seq = file.write(new_content)
        
    return seq

In [187]:
#combine both of the above functions

def fasta_to_descriptors(fasta:str, descriptors=[]):

    #remove description from fasta file
    remove_fasta_description(fasta)
    
    #return protein descriptors
    return get_protein_descriptors(fasta, descriptors=descriptors)

In [188]:
#need to figure out how to pass fasta for both descriptors and append to one dataframe
#set this back to what it was originally. Gonna calculate difference for each descriptors

# def create_new_dataframe(dataframe, fasta_files:list, descriptors=[]):
#     """
#     Creates new dataframe with descriptors added.
    
#     Parameters
#     ----------
#     Pandas dataframe, list of descriptors as strings, output file name.

#     Returns
#     -------
#     Dataframe including vector(s) of descriptors
#     """
    
#     #descriptors for meso sequence
#     fasta = get_fasta_from_dataframe(dataframe, fasta_files[0], fasta_files[1])
    
#     #create feature dictionary for sequence a
#     feature_dict_a = get_protein_descriptors(fasta[0], 'Meso', descriptors)
#     print(feature_dict_a)
    
#     #create feature dictionary for sequence b
#     feature_dict_b = get_protein_descriptors(fasta[1], 'Thermo', descriptors)
    
#     df = dataframe.reset_index()
    
#     #merge meso sequences
#     for desc in descriptors:
        
#         feature_dict_a[desc+'_Meso'].index = feature_dict_a[desc+'_Meso'].index.astype(int)
#         features_a = feature_dict_a[desc+'_Meso'].reset_index()
        
#     df = pd.merge(df, features_a, how='outer', left_index=True, right_index=True)
    
#     #merge thermo sequences
# #     for desc in descriptors:
        
# #         feature_dict_b[desc+'_Thermo'].index = feature_dict_b[desc+'_Thermo'].index.astype(int)
# #         features_b = feature_dict_b[desc+'_Thermo'].reset_index()
        
# #     df = pd.merge(df, features_b, how='outer', left_index=True, right_index=True)
        
#     return df

Let's see how long generating some of the descriptors takes:

In [189]:
df

,index_x,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,...,AAC_T,AAC_V,AAC_W,AAC_Y,index,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
0,0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,...,-0.000961,0.062242,-0.000192,-0.010478,48641291,0.065318,-0.025810,0.011583,-0.017639,-0.033452
1,1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,...,-0.016081,0.018254,0.000072,0.000507,92992745,-0.014143,0.009525,-0.010051,0.032651,-0.017982
2,2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,...,-0.007743,0.011808,0.002516,-0.018293,157628663,0.087205,-0.020132,0.040650,0.006291,-0.114015
3,3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,...,-0.011915,0.019668,0.018762,-0.015251,136708305,0.015966,0.014457,0.001080,-0.011359,-0.020144
4,4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,...,-0.005303,0.042045,0.022348,0.017045,133672542,0.091667,-0.027273,-0.084470,-0.031061,0.051136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,6343,6495,0.387574,0.367978,0.360882,336,0.943820,337,0.910811,588,...,-0.004616,0.011388,-0.006043,0.003811,55489429,0.004783,-0.017871,-0.019040,0.008624,0.023504
6344,6344,6496,0.488462,0.488462,0.455197,258,0.992308,263,0.882550,619,...,-0.011693,-0.033867,-0.001058,0.000903,172293605,-0.039210,0.011306,-0.000800,0.021631,0.007073
6345,6345,6497,0.383838,0.376238,0.374384,296,0.976898,299,0.977124,436,...,0.029023,-0.056753,-0.003397,-0.013266,47082975,-0.035009,-0.017019,0.008801,0.002362,0.040866
6346,6346,6498,0.769231,0.769231,0.769231,78,1.000000,78,1.000000,258,...,-0.051282,0.000000,0.000000,0.000000,154513027,0.064103,-0.012821,0.012821,-0.025641,-0.038462


In [190]:
df
df['protein_match'].value_counts()

Yes    5243
No     1105
Name: protein_match, dtype: int64

In [191]:
#iFeature properties

protein = iFeatureOmegaCLI.iProtein('meso_50k.fasta')
protein.display_feature_types()


        ----- Available feature types ------        
        
        AAC                                                Amino acid composition
        EAAC                                               Enhanced amino acid composition
        CKSAAP type 1                                      Composition of k-spaced amino acid pairs type 1 - normalized
        CKSAAP type 2                                      Composition of k-spaced amino acid pairs type 2 - raw count
        DPC type 1                                         Dipeptide composition type 1 - normalized
        DPC type 2                                         Dipeptide composition type 2 - raw count
        TPC type 1                                         Tripeptide composition type 1 - normalized
        TPC type 2                                         Tripeptide composition type 1 - raw count
        CTDC                                               Composition
        CTDT                                      

In [192]:
"""
this list comes from a combination of reading through the features and determining which might be useful
and timing some of the feature generations. those that took more than 30ish seconds were eliminated
Also removed those that have really high dimensionality (>4000)
"""

feature_list = ['AAC', 'GAAC', 'DistancePair',
               'DPC type 1', 'CTDC', 'CTDT', 'CTDD', 'CTriad',
                'CKSAAGP type 1', 'PseKRAAC type 1', 'PseKRAAC type 2', 'PseKRAAC type 3A',
                'PseKRAAC type 3B', 'PseKRAAC type 4', 'PseKRAAC type 5',
                'APAAC', 'QSOrder']

In [193]:
import time

In [194]:
def time_feature_generation(dataframe, file_name:str, descriptor:str):
    
    """
    Times how long it takes to generate a specific descriptor.
    
    Parameters
    ----------
    Pandas dataframe, fasta file name, descriptor name (string).

    Returns
    -------
    Time to generate descriptor.
    """

    # Record the current time
    start_time = time.time()

    # Code to be timed goes here
    create_new_dataframe(dataframe, file_name, descriptors=[descriptor])

    # Record the time again and calculate the elapsed time
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Print the elapsed time
    return elapsed_time

In [195]:
time_feature_generation(df, 'meso_50k.fasta', 'QSOrder')

File imported successfully.
File imported successfully.


<ipython-input-184-97c201eec18b>:50: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(


11.930752992630005

In [196]:
def time_feature_dict(dataframe, file_name:str, feature_list):
    
    """
    Iterates through dictionary of protein descriptors and returns the time it takes to run the code.
    
    Parameters
    ----------
    Pandas dataframe, fasta file name, descriptors (list of strings).

    Returns
    -------
    Dictionary of descriptors and time to generate them.
    """
    
    protein = iFeatureOmegaCLI.iProtein(file_name)
    
    time_dict = {}
    
    for feature in feature_list:
        time_dict[feature] = time_feature_generation(dataframe, file_name, feature)
    
    return time_dict

In [197]:
# time_feature_dict(df, 'meso_50k.fasta', feature_list)

In [198]:
df['Jaccard_Score']

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
6343    0.5
6344    1.0
6345    1.0
6346    1.0
6347    0.2
Name: Jaccard_Score, Length: 6348, dtype: float64

In [199]:
df

,index_x,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,...,AAC_T,AAC_V,AAC_W,AAC_Y,index,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
0,0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,...,-0.000961,0.062242,-0.000192,-0.010478,48641291,0.065318,-0.025810,0.011583,-0.017639,-0.033452
1,1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,...,-0.016081,0.018254,0.000072,0.000507,92992745,-0.014143,0.009525,-0.010051,0.032651,-0.017982
2,2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,...,-0.007743,0.011808,0.002516,-0.018293,157628663,0.087205,-0.020132,0.040650,0.006291,-0.114015
3,3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,...,-0.011915,0.019668,0.018762,-0.015251,136708305,0.015966,0.014457,0.001080,-0.011359,-0.020144
4,4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,...,-0.005303,0.042045,0.022348,0.017045,133672542,0.091667,-0.027273,-0.084470,-0.031061,0.051136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,6343,6495,0.387574,0.367978,0.360882,336,0.943820,337,0.910811,588,...,-0.004616,0.011388,-0.006043,0.003811,55489429,0.004783,-0.017871,-0.019040,0.008624,0.023504
6344,6344,6496,0.488462,0.488462,0.455197,258,0.992308,263,0.882550,619,...,-0.011693,-0.033867,-0.001058,0.000903,172293605,-0.039210,0.011306,-0.000800,0.021631,0.007073
6345,6345,6497,0.383838,0.376238,0.374384,296,0.976898,299,0.977124,436,...,0.029023,-0.056753,-0.003397,-0.013266,47082975,-0.035009,-0.017019,0.008801,0.002362,0.040866
6346,6346,6498,0.769231,0.769231,0.769231,78,1.000000,78,1.000000,258,...,-0.051282,0.000000,0.000000,0.000000,154513027,0.064103,-0.012821,0.012821,-0.025641,-0.038462


In [200]:
df.columns

Index(['index_x', 'Unnamed: 0', 'local_gap_compressed_percent_id',
       'scaled_local_query_percent_id', 'scaled_local_symmetric_percent_id',
       'query_align_len', 'query_align_cov', 'subject_align_len',
       'subject_align_cov', 'bit_score', 'thermo_index', 'meso_index',
       'prot_pair_index', 'meso_protein_int_index', 'thermo_protein_int_index',
       'taxa_pair_index', 'local_gap_compressed_percent_id_16s',
       'scaled_local_query_percent_id_16s',
       'scaled_local_symmetric_percent_id_16s', 'query_align_cov_16s',
       'subject_align_cov_16s', 'bit_score_16s', 'm_ogt', 't_ogt',
       'ogt_difference', 'm_protein_seq', 't_protein_seq', 'm_protein_desc',
       't_protein_desc', 'm_protein_len', 't_protein_len', 'Jaccard_Score',
       'protein_match', 'index_y', 'AAC_A', 'AAC_C', 'AAC_D', 'AAC_E', 'AAC_F',
       'AAC_G', 'AAC_H', 'AAC_I', 'AAC_K', 'AAC_L', 'AAC_M', 'AAC_N', 'AAC_P',
       'AAC_Q', 'AAC_R', 'AAC_S', 'AAC_T', 'AAC_V', 'AAC_W', 'AAC_Y', 'index',
 

# Scratch work for above function

In [201]:
# sequence = "meso_input_copy.fasta.txt"
# string_to_remove = "<unknown description>"

# with open(sequence, "r") as file:
#     content = file.read()
    
# # Remove the string
# new_content = content.replace(string_to_remove, "")

# with open(sequence, "w") as file:
#     file.write(new_content)


In [202]:
df['t_protein_len'].describe()

count    6348.000000
mean      264.121141
std        67.865076
min        35.000000
25%       229.000000
50%       262.000000
75%       312.000000
max       400.000000
Name: t_protein_len, dtype: float64

The bit-score provides a better rule-of-thumb for inferring homology. For average length proteins, a bit score of 50 is almost always significant. A bit score of 40 is only significant (E() < 0.001) in searches of protein databases with fewer than 7000 entries. Increasing the score by 10 bits increases the significance 210=1000-fold, so 50 bits would be significant in a database with less than 7 million entries (10 times SwissProt, and within a factor of 3 of the largest protein databases). Thus, the NCBI Blast web site uses a color code of blue for alignment with scores between 40–50 bits; and green for scores between 50–80 bits. In the yeast vs human example, the alignments with less than 20% identity had scores ranging from 55 – 170 bits. Except for very long proteins and very large databases, 50 bits of similarity score will always be statistically significant and is a much better rule-of-thumb for inferring homology in protein alignments.

Pearson et al., 2013: An Introduction to Sequence Similarity (“Homology”) Searching

In [203]:
df['protein_match'].value_counts()

Yes    5243
No     1105
Name: protein_match, dtype: int64

In [204]:
df

,index_x,Unnamed: 0,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,...,AAC_T,AAC_V,AAC_W,AAC_Y,index,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
0,0,0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,...,-0.000961,0.062242,-0.000192,-0.010478,48641291,0.065318,-0.025810,0.011583,-0.017639,-0.033452
1,1,1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,...,-0.016081,0.018254,0.000072,0.000507,92992745,-0.014143,0.009525,-0.010051,0.032651,-0.017982
2,2,2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,...,-0.007743,0.011808,0.002516,-0.018293,157628663,0.087205,-0.020132,0.040650,0.006291,-0.114015
3,3,3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,...,-0.011915,0.019668,0.018762,-0.015251,136708305,0.015966,0.014457,0.001080,-0.011359,-0.020144
4,4,4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,...,-0.005303,0.042045,0.022348,0.017045,133672542,0.091667,-0.027273,-0.084470,-0.031061,0.051136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6343,6343,6495,0.387574,0.367978,0.360882,336,0.943820,337,0.910811,588,...,-0.004616,0.011388,-0.006043,0.003811,55489429,0.004783,-0.017871,-0.019040,0.008624,0.023504
6344,6344,6496,0.488462,0.488462,0.455197,258,0.992308,263,0.882550,619,...,-0.011693,-0.033867,-0.001058,0.000903,172293605,-0.039210,0.011306,-0.000800,0.021631,0.007073
6345,6345,6497,0.383838,0.376238,0.374384,296,0.976898,299,0.977124,436,...,0.029023,-0.056753,-0.003397,-0.013266,47082975,-0.035009,-0.017019,0.008801,0.002362,0.040866
6346,6346,6498,0.769231,0.769231,0.769231,78,1.000000,78,1.000000,258,...,-0.051282,0.000000,0.000000,0.000000,154513027,0.064103,-0.012821,0.012821,-0.025641,-0.038462


In [205]:
df['index_x']

0          0
1          1
2          2
3          3
4          4
        ... 
6343    6343
6344    6344
6345    6345
6346    6346
6347    6347
Name: index_x, Length: 6348, dtype: int64

In [206]:
#get rid of stuff that isn't quantitative

df = df.drop(columns = (['Unnamed: 0', 'index_x', 'thermo_index',
                         'm_protein_seq', 't_protein_seq', 'm_protein_desc', 't_protein_desc']))

In [207]:
df.head()

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,meso_index,prot_pair_index,...,AAC_T,AAC_V,AAC_W,AAC_Y,index,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
0,0.287582,0.217822,0.215686,160,0.792079,152,0.737864,131,12897,48641291,...,-0.000961,0.062242,-0.000192,-0.010478,48641291,0.065318,-0.025810,0.011583,-0.017639,-0.033452
1,0.319635,0.295359,0.297872,218,0.919831,226,0.969957,282,13026,92992745,...,-0.016081,0.018254,0.000072,0.000507,92992745,-0.014143,0.009525,-0.010051,0.032651,-0.017982
2,0.279621,0.234127,0.218924,211,0.837302,210,0.731707,96,8203,157628663,...,-0.007743,0.011808,0.002516,-0.018293,157628663,0.087205,-0.020132,0.040650,0.006291,-0.114015
3,0.327273,0.200743,0.214712,166,0.617100,163,0.696581,175,3340,136708305,...,-0.011915,0.019668,0.018762,-0.015251,136708305,0.015966,0.014457,0.001080,-0.011359,-0.020144
4,0.338710,0.318182,0.287671,60,0.909091,71,0.887500,61,14020,133672542,...,-0.005303,0.042045,0.022348,0.017045,133672542,0.091667,-0.027273,-0.084470,-0.031061,0.051136


From pairplot below, it looks like query_align_cov_16s and subject_align_cov_16s dont correlate with target. Let's remove them.

In [208]:
df = df.drop(columns = ['query_align_cov_16s', 'subject_align_cov_16s'])

In [209]:
# sns.pairplot(df.sample(500),height = 4)

In [210]:
df.describe()

,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,meso_index,prot_pair_index,...,AAC_T,AAC_V,AAC_W,AAC_Y,index,GAAC_alphatic,GAAC_aromatic,GAAC_postivecharge,GAAC_negativecharge,GAAC_uncharge
count,6348.000000,6348.000000,6348.000000,6348.000000,6348.000000,6348.000000,6348.000000,6348.000000,6348.000000,6.348000e+03,...,6348.000000,6348.000000,6348.000000,6348.000000,6.348000e+03,6348.000000,6348.000000,6348.000000,6348.000000,6348.000000
mean,0.348174,0.276047,0.270881,208.339162,0.792409,207.604600,0.773547,233.339162,8098.207152,9.059579e+07,...,0.002973,-0.000936,-0.000822,-0.000874,9.059579e+07,0.004287,-0.001298,-0.008658,-0.002680,0.008348
std,0.096390,0.113913,0.115180,63.261682,0.145697,63.264993,0.175379,186.940699,4565.102004,5.250268e+07,...,0.020149,0.024541,0.008551,0.011881,5.250268e+07,0.045827,0.019876,0.023366,0.022492,0.038984
min,0.193431,0.106383,0.066929,23.000000,0.327935,23.000000,0.083916,37.000000,5.000000,8.858000e+03,...,-0.081561,-0.109956,-0.057143,-0.136364,8.858000e+03,-0.177149,-0.090909,-0.152017,-0.211716,-0.174242
25%,0.292571,0.205948,0.203093,176.000000,0.672716,176.000000,0.657086,121.000000,4243.250000,4.530101e+07,...,-0.009939,-0.016708,-0.005470,-0.008303,4.530101e+07,-0.026531,-0.013625,-0.022834,-0.016520,-0.016369
50%,0.323720,0.250000,0.244565,211.000000,0.811727,210.000000,0.805808,188.000000,8046.000000,9.085625e+07,...,0.002729,-0.000799,-0.000080,-0.000484,9.085625e+07,0.003705,-0.001190,-0.007898,-0.002856,0.007810
75%,0.366709,0.306723,0.300314,241.000000,0.925000,240.000000,0.924258,270.000000,12069.000000,1.360285e+08,...,0.015633,0.014542,0.004072,0.006596,1.360285e+08,0.034317,0.011286,0.006209,0.011004,0.033613
max,0.959350,0.951613,0.955466,398.000000,1.000000,398.000000,1.000000,1565.000000,16345.000000,1.814286e+08,...,0.100446,0.113636,0.043723,0.061224,1.814286e+08,0.295455,0.086914,0.098889,0.106208,0.193513


Split data into dev and test, and then split that into train and validation.

In [211]:
#drop columns that don't exihibit signficant pearson correlation with bit_score

df = df.drop(columns = ['meso_index', 'meso_protein_int_index', 'local_gap_compressed_percent_id_16s', 
                        'scaled_local_query_percent_id_16s', 'scaled_local_symmetric_percent_id_16s',
                       'bit_score_16s', 'm_ogt', 't_ogt', 'taxa_pair_index', 'thermo_protein_int_index'
                       , 'prot_pair_index', 'ogt_difference'])

In [219]:
df = df.drop(columns=['protein_match'])

In [220]:
#choosing 80/20 split instead of 85/15 because of volume of data

dev, test = sklearn.model_selection.train_test_split(df, test_size=0.15, random_state=1)

train, val = sklearn.model_selection.train_test_split(dev, test_size=0.15, random_state=1)

print(dev.shape)
print(test.shape)
print(train.shape)
print(val.shape)

(5395, 38)
(953, 38)
(4585, 38)
(810, 38)


In [221]:
#ID target and features, separate into separate arrays

target = 'Jaccard_Score'
input_features = [columns for columns in df]
input_features.remove(target)

In [222]:
print(input_features)
print(target)

['local_gap_compressed_percent_id', 'scaled_local_query_percent_id', 'scaled_local_symmetric_percent_id', 'query_align_len', 'query_align_cov', 'subject_align_len', 'subject_align_cov', 'bit_score', 'm_protein_len', 't_protein_len', 'index_y', 'AAC_A', 'AAC_C', 'AAC_D', 'AAC_E', 'AAC_F', 'AAC_G', 'AAC_H', 'AAC_I', 'AAC_K', 'AAC_L', 'AAC_M', 'AAC_N', 'AAC_P', 'AAC_Q', 'AAC_R', 'AAC_S', 'AAC_T', 'AAC_V', 'AAC_W', 'AAC_Y', 'index', 'GAAC_alphatic', 'GAAC_aromatic', 'GAAC_postivecharge', 'GAAC_negativecharge', 'GAAC_uncharge']
Jaccard_Score


In [223]:
#split X and y

dev_X = dev[input_features].values
test_X = test[input_features].values

dev_y = dev[target].values.reshape(-1,1)
test_y = test[target].values.reshape(-1,1)  

print(dev_X.shape, test_X.shape, dev_y.shape, test_y.shape)

(5395, 37) (953, 37) (5395, 1) (953, 1)


In [224]:
#same thing for training and validation data

train_X = train[input_features].values
val_X = val[input_features].values

train_y = train[target].values.reshape(-1,1)
val_y = val[target].values.reshape(-1,1) 

Scale the data

In [225]:
scaler = sklearn.preprocessing.StandardScaler()
dev_X = scaler.fit_transform(dev_X)
test_X = scaler.fit_transform(test_X)
train_X = scaler.fit_transform(train_X)
val_X = scaler.fit_transform(val_X)

Train the model

In [227]:
#Random Forest

model = sklearn.ensemble.RandomForestRegressor()

model.fit(train_X, train_y.ravel())

RandomForestRegressor()

Test the model, report relevant statistics

In [228]:
score = model.score(val_X, val_y)
print('Model score is: {}'.format(score))

preds = model.predict(test_X)
print(preds)

Model score is: 0.3669055969884438
[0.60390795 0.96       0.6739881  0.43842857 0.52303571 0.61281602
 0.66175938 0.80086111 0.42849964 0.6289127  0.5912619  0.69910714
 0.38770635 0.66678968 0.46654365 0.48885442 0.26077778 0.44075577
 0.91191667 0.48374603 0.76127778 0.05313492 0.76395238 0.40683442
 0.96416667 0.82580159 0.28683333 0.20842857 0.67293651 0.65707143
 0.81506349 0.81621429 0.69366667 0.74704762 0.91341667 0.82391667
 0.87766667 0.7987381  0.64969322 0.59069872 0.77635317 0.97029487
 0.41793162 0.77604762 0.74879762 0.76415995 0.75471429 0.45893362
 0.56573918 0.48990476 0.94983333 0.60159127 0.37009524 0.68502778
 0.42476465 0.85961111 0.9072381  0.87058333 0.71476623 0.73829437
 0.50654906 0.77716667 0.74466667 0.85607937 0.63891667 0.95821429
 0.74572619 0.87158333 0.7576746  0.953      0.5851562  0.80288889
 0.62020238 0.65015751 0.4565174  0.4435689  0.81195635 0.66874603
 0.64269048 0.55076984 0.53645665 0.42031566 0.85414286 0.91166667
 0.73866667 0.86316667 0.54

In [78]:
#confusion matrix

confusion_matrix = sklearn.metrics.confusion_matrix(preds, test_y)
sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix).plot()

Convert the above code into functions.

In [79]:
def train_model(dataframe, columns = [],  target = []):
    """
    Takes dataframe and splits it into a training and testing set. 
    Note: Data is called train and test, but this test set is currently
    closer to a validation set. Keeping nomenclature to keep model robust.
    Trains a KNN classifier model with selected data.
    
    Params
    ----------
    dataframe: Pandas dataframe 
    columns: list of strings, representing input features
    target: list of strings, representing target feature(s)

    Returns
    -------
    -Sk-learn model object
    -train data (features)
    -train data (target)
    -validation data (features)
    -validation data (target)
    """
    #split data
    dev, test = sklearn.model_selection.train_test_split(dataframe, test_size=0.15, random_state=1)
    
    #test input arguments
    assert "pandas.core.frame.DataFrame" in str(type(dev))
    assert "pandas.core.frame.DataFrame" in str(type(test))
    assert "str" in str(type(columns[0]))
    assert "str" in str(type(target[0]))
   
    #split into input and output feature(s)
    dev_X = dev[columns].values
    test_X = test[columns].values

    dev_y = dev[target].values.reshape(-1,1)
    test_y = test[target].values.reshape(-1,1)
    
    #scale data
    scaler = sklearn.preprocessing.StandardScaler()
    dev_X = scaler.fit_transform(dev_X)
    test_X = scaler.fit_transform(test_X)
    
    #train model
    model = sklearn.ensemble.RandomForestClassifier()
    model = model.fit(dev_X, dev_y.ravel())
    
    return model, dev_X, dev_y, test_X, test_y
    

In [80]:
model, dev_X, dev_y, test_X, test_y = train_model(df, columns = input_features, 
                                          target='protein_match')

In [81]:
class TestModelTraining(unittest.TestCase):
    
    def test_invalid_inputs(self):
    
        #test that input data type is correct
        
        try:
            train_model([1,2,3], columns = 'string', target = 'string')
            self.assertTrue(False)
        except AssertionError:
            self.assertTrue(True)
    
#     def test_input_distro(self):
        
#         #test that dev and test features have similar Jensen Shannon Distribution
        
#         JSD = (
#             scipy.stats.bootstrap((train_reg(split_data(df)[0], split_data(df)[1],
#                                 columns = input_features, target=target)[1], train_reg(split_data(df)[0], 
#                                 split_data(df)[1], columns = input_features, target=target)[3]), 
#                                   JSD_dev_and_test, n_resamples=1000, 
#                                   batch=5, method='percentile')
#         )

#         div = JSD.confidence_interval[1]
        
#         #asserts that the divergence between data sets is sufficiently low
#         assert abs(div) < 0.3, "Warning! High JSD between dev and test set!"
        
    def test_output_format(self):
        
        #asserts that function returns 4 objects to be assigned to pearson_corr, model, test_X, test_y
        assert len(train_model(df, columns = input_features, 
                                          target = target)) == 5
        

suite = unittest.TestLoader().loadTestsFromTestCase(TestModelTraining)
_ = unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 14.866s

OK


In [46]:
#need to fix this

def evaluate_model(model, test_X, test_y):
    
    """
    Takes a trained model and test data and tests the model.
    
    Params
    ----------
    model: sklearn.neighbors.KNeighborsClassifier
    test_X: numpy array
    test_y: numpy array

    Returns
    -------
    Vector of predictions based on the model (numpy array)
    """
    
    #test input arguments
    assert "sklearn" in str(type(model))
    assert "numpy.ndarray" in str(type(test_X))
    assert "numpy.ndarray" in str(type(test_y))
    
    preds = model.predict(test_X)
    
    return preds

In [47]:
preds = evaluate_model(model, test_X, test_y)
preds

array([False, False, False, ..., False,  True, False])

In [48]:
class TestModelPerformance(unittest.TestCase):

    def test_asserts(self):
        model, _, _, test_X, test_y = train_model(
            df, columns=input_features, target='protein_match'
        )
        # assert that input types are correct
        with self.assertRaises(AssertionError):
            evaluate_model(model, [1, 2, 3], test_y)
            
    def test_model_output(self):
        model, _, _, test_X, test_y = train_model(
            df, columns=input_features, target='protein_match'
        )
        # assert output type is correct
        output = evaluate_model(model, test_X, test_y)
        self.assertIsInstance(output, np.ndarray)
        
    def test_pred_dimension(self):
        model, _, _, test_X, test_y = train_model(df, 
            columns=input_features, target='protein_match'
        )
        # want to check that the number of predictions is equal to the number of test examples
        preds = evaluate_model(model, test_X, test_y)
        self.assertEqual(len(test_y), len(preds))

suite = unittest.TestLoader().loadTestsFromTestCase(TestModelPerformance)
_ = unittest.TextTestRunner().run(suite)


...
----------------------------------------------------------------------
Ran 3 tests in 236.519s

OK


A good test here can be adding outliers and seeing how much the score and predictions change (probably for the above function).

In [49]:
def plot_model(model, test_X, test_y):
    """
    Takes a test KNN Classifier model and plots the confusion matrix.
    
    Params
    ----------
    model: sklearn.neighbors.KNeighborsClassifier
    test_X: numpy array
    test_y: numpy array

    Returns
    -------
    -Confusion predictions vs. observations
    -Model score
    """
    
    #test input arguments
    assert "sklearn" in str(type(model))
    assert "numpy.ndarray" in str(type(test_X))
    assert "numpy.ndarray" in str(type(test_y))
    
    score = model.score(test_X, test_y)
    preds = evaluate_model(model, test_X, test_y)
   
    # plot confusion matrix
    confusion_matrix = sklearn.metrics.confusion_matrix(preds, test_y)
    cm_plot = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix)
    
    cm_plot.plot(cmap=plt.cm.Blues)
    cm_plot.ax_.set_title('Confusion Matrix')
    
    return score
    

In [50]:
#make some appropriate display labels here

plot_model(model, test_X, test_y)

0.9904

In [51]:
#wrapper function

def RF_wrapper(dataframe):
    """
    Takes dataframe and runs it through kNN model.
    
    Params
    ----------
    dataframe: Pandas dataframe 

    Returns
    -------
    -Target feature predictions
    -Parity plot
    """
    
    assert 'pandas.core.frame.DataFrame' in str(type(dataframe))

    #user inputs target feature
    target = 'protein_match'
    
    #define input features
    input_features = [columns for columns in dataframe]
    
    input_features.remove(target)

    #train the model based off data split
    model, dev_X, dev_y, test_X, test_y = train_model(
                                        dataframe, columns=input_features, 
                                        target=target
                                                   )
    
    #test the model and return predictions
    preds = evaluate_model(model, test_X, test_y)

    #plot the results of the model
    plot_model(model, test_X, test_y)
    
    return preds

In [52]:
RF_wrapper(df)

array([False, False, False, ..., False,  True, False])

In [53]:
class TestWrapper(unittest.TestCase):
    
    def test_wrapper_input(self):
        #test that input data type is correct
        try:
            RF_wrapper([1,2,3])
            self.assertTrue(False)
        except AssertionError:
            self.assertTrue(True)

    def test_wrapper_output(self):
        model, _, _, test_X, test_y = train_model(
            df, 
            columns=input_features, target='protein_match'
        )
        # assert output type is correct
        output = evaluate_model(model, test_X, test_y)
        self.assertIsInstance(output, np.ndarray)
        
    def test_output_dimension(self):
        model, _, _, test_X, test_y = train_model(
            df, 
            columns=input_features, target='protein_match'
        )
        # want to check that the number of predictions is equal to the number of test examples
        preds = evaluate_model(model, test_X, test_y)
        self.assertEqual(len(test_y), len(preds))
        

suite = unittest.TestLoader().loadTestsFromTestCase(TestWrapper)
_ = unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 158.947s

OK


**IGNORE BELOW**

In [54]:
# #plot empirical distribution of scaled_local_query_percent_id

# target = df['protein_match']
# #create histplot
# fig, ax = plt.subplots()

# sns.histplot(data = df, x = target)

# ax.set_title('Sample Histogram', fontsize=16)
# ax.set_xlabel('Class', fontsize=14)
# ax.set_ylabel('Count', fontsize=14)

Things I've tried to improve model: 

1. drop bit scores over 1000
2. Switch to 85/15 train/test split
3. Lasso regression - not great
4. Ridge regression - R2 = 0.87, about the same as normal Linear regression
5. KNN regression (n_neighbors optimized at 8) gives best result, R2 = ~0.93 --> 
*got it up to .942 by removing a few features
6. Decision tree regressor was slightly worse than KNN
7. Input DT classifier and RF classifier. RF classifier has best performance
8. Changed target to binary protein function match